# TMDB 5000 Movie Dataset으로 장르 기반 추천 시스템 만들기
- https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv
---
- 장르에 있는 텍스트를 벡터화 후 텍스트 간의 유사도를 구해 가까운 순서대로 정렬
- 유사도 척도 = 코사인(Cosine) 유사도
- 보험, 예금 상품의 특징을 추천하여 추천하는데 사용될 수 있음

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [22]:
data = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv')
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [10]:
dict_cols = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']

In [52]:
data.isna().sum() / len(data) * 100

budget                   0.000000
genres                   0.000000
homepage                64.355611
id                       0.000000
keywords                 0.000000
original_language        0.000000
original_title           0.000000
overview                 0.062461
popularity               0.000000
production_companies     0.000000
production_countries     0.000000
release_date             0.020820
revenue                  0.000000
runtime                  0.041641
spoken_languages         0.000000
status                   0.000000
tagline                 17.572351
title                    0.000000
vote_average             0.000000
vote_count               0.000000
dtype: float64

In [53]:
print(type(data['genres'][0]))
data['genres'][0]

<class 'str'>


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
import json

In [55]:
result = []
for item in json.loads(data['genres'][0]):
    print(item['name'])
    result.append(item['name'])

result = " ".join(result)
result

Action
Adventure
Fantasy
Science Fiction


'Action Adventure Fantasy Science Fiction'

In [35]:
# 딕셔너리(JSON) 컬럼 언팩 함수
def dict_unpack_detail(data):
    print('='*10, 'apply(dict_unpack, axis=1)로 들어온 data', '='*10)
    print(data, end='\n\n')
    for col in dict_cols:
        print('='*10, f"data[{col}]", '='*10)
        print(json.loads(data[col]), end='\n\n')
        temp = []
        for i in json.loads(data[col]):
            temp.append(i.get('name', 'None'))
        temp = " ".join(temp)
        data[col] = temp
        print('='*5, f"data[{col}]에서 name의 value만 추출해 문자열로 만든 결과", '='*10)
        print(data)
    return data


In [36]:
# 딕셔너리(JSON) 컬럼 언팩 함수
def dict_unpack(data):
    for col in dict_cols:
        temp = []
        for i in json.loads(data[col]):
            temp.append(i.get('name', 'None'))
        temp = " ".join(temp)
        data[col] = temp
    return data

In [37]:
data2 = data.iloc[:2, :].apply(dict_unpack_detail, axis=1)
data2

========== apply(dict_unpack, axis=1)로 들어온 data ==========
budget                                                          237000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                                      http://www.avatarmovie.com/
id                                                                  19995
keywords                [{"id": 1463, "name": "culture clash"}, {"id":...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                                              

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [38]:
data = data.apply(dict_unpack, axis=1)

In [39]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [40]:
data = data[['original_title', 'genres', 'popularity', 'runtime', 'vote_average', 'vote_count']]
data.head()

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,139.082615,169.0,6.9,4500
2,Spectre,Action Adventure Crime,107.376788,148.0,6.3,4466
3,The Dark Knight Rises,Action Crime Drama Thriller,112.312950,165.0,7.6,9106
4,John Carter,Action Adventure Science Fiction,43.926995,132.0,6.1,2124


- 장르 컬럼의 텍스트를 숫자로 벡터화
- 코사인 유사도를 구해 비슷한 장르의 행 검색
- 유사도가 높은 행을 기준으로 내림차순 정렬

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
cv = CountVectorizer(ngram_range=(1,2))
genres_mat = cv.fit_transform(data['genres'])
# genres_mat.shape
# cv.get_feature_names_out()

genres_mat_df = pd.DataFrame(genres_mat)
genres_mat_df

,0
0,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
1,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
2,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
3,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
4,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
...,...
4798,"(np.int32(0), np.int32(0))\t1\n (np.int32(0..."
4799,"(np.int32(0), np.int32(44))\t1\n (np.int32(..."
4800,"(np.int32(0), np.int32(90))\t1\n (np.int32(..."
4801,


- 코사인 유사도를 이용해 genres_mat의 유사도 산출 후 genres_sim 생성
- 비슷한 장르의 영화를 찾기 위해 계산
- 코사인 유사도는 주로 문자, 문서의 유사도를 계산하는데 사용됨

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
genres_sim = cosine_similarity(genres_mat, genres_mat)
print(genres_sim.shape)

(4803, 4803)


In [48]:
genres_sim_df = pd.DataFrame(genres_sim)
genres_sim_df

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.596285,0.447214,0.125988,0.755929,0.596285,0.0,0.755929,0.447214,0.745356,...,0.000000,0.000000,0.000000,0.377964,0.000000,0.149071,0.0000,0.000000,0.0,0.0
1,0.596285,1.000000,0.400000,0.169031,0.338062,0.800000,0.0,0.338062,0.600000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.0,0.0
2,0.447214,0.400000,1.000000,0.338062,0.507093,0.600000,0.0,0.507093,0.200000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.0000,0.000000,0.0,0.0
3,0.125988,0.169031,0.338062,1.000000,0.142857,0.169031,0.0,0.142857,0.000000,0.169031,...,0.377964,0.169031,0.377964,0.428571,0.218218,0.676123,0.0000,0.125988,0.0,0.0
4,0.755929,0.338062,0.507093,0.142857,1.000000,0.507093,0.0,1.000000,0.169031,0.507093,...,0.000000,0.000000,0.000000,0.428571,0.000000,0.169031,0.0000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.149071,0.200000,0.400000,0.676123,0.169031,0.200000,0.0,0.169031,0.000000,0.200000,...,0.000000,0.200000,0.000000,0.169031,0.258199,1.000000,0.0000,0.000000,0.0,0.0
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.258199,0.000000,0.000000,0.000000,0.000000,1.0000,0.384900,0.0,0.0
4800,0.000000,0.000000,0.000000,0.125988,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.333333,0.149071,0.333333,0.125988,0.000000,0.000000,0.3849,1.000000,0.0,0.0
4801,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0


In [49]:
sorted_genres_sim = genres_sim.argsort()[:, ::-1]

In [50]:
sorted_genres_sim[:1]

array([[   0,   14,  870, ..., 2041, 3480, 2038]])

In [51]:
data.iloc[0, :]

original_title                                      Avatar
genres            Action Adventure Fantasy Science Fiction
popularity                                      150.437577
runtime                                              162.0
vote_average                                           7.2
vote_count                                           11800
Name: 0, dtype: object

In [53]:
data.iloc[870, :]

original_title                                 Superman II
genres            Action Adventure Fantasy Science Fiction
popularity                                       30.515175
runtime                                              127.0
vote_average                                           6.5
vote_count                                             629
Name: 870, dtype: object

In [54]:
data.iloc[2315, :]

original_title    Borat: Cultural Learnings of America for Make ...
genres                                                       Comedy
popularity                                                29.487784
runtime                                                        82.0
vote_average                                                    6.5
vote_count                                                     1579
Name: 2315, dtype: object

# 영화 이름과 추천 받을 개수를 입력받아 출력하는 함수

In [55]:
data.columns

Index(['original_title', 'genres', 'popularity', 'runtime', 'vote_average',
       'vote_count'],
      dtype='object')

In [56]:
data[data['original_title'] == 'Avatar'].index.values[0]

np.int64(0)

In [57]:
sorted_genres_sim[0, 1:11]

array([  14,  870,  813, 3494,   46, 1652, 1296,  419,  420, 1932])

In [58]:
data.iloc[sorted_genres_sim[0,1:11], :]

,original_title,genres,popularity,runtime,vote_average,vote_count
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
3494,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,1.478505,107.0,4.6,17
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
1652,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,21.677732,85.0,2.9,462
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
419,Jumper,Adventure Fantasy Science Fiction,21.218000,88.0,5.9,1799
420,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,58.579760,120.0,6.5,1527
1932,Sheena,Action Adventure Comedy Fantasy Science Fiction,4.020194,117.0,5.0,22


In [59]:
def find_sim_movie(data, sorted_genres_sim, title_name, top_n=10):
    title_movie = data[data['original_title'] == title_name]
    title_index = title_movie.index.values[0]
    similer_idxs = sorted_genres_sim[title_index, 1:top_n+1]
    return data.iloc[similer_idxs]
    

In [60]:
find_sim_movie(data, sorted_genres_sim, "Sheena", 10)

,original_title,genres,popularity,runtime,vote_average,vote_count
859,Thunderbirds,Action Adventure Comedy Family Fantasy Science...,9.278750,95.0,4.2,91
2069,Teenage Mutant Ninja Turtles III,Action Adventure Comedy Family Fantasy Science...,10.207065,96.0,5.1,184
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
1936,Tank Girl,Action Comedy Fantasy Science Fiction,8.186936,98.0,5.5,138
342,Men in Black,Action Adventure Comedy Science Fiction,104.121555,98.0,6.9,4412
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800


In [61]:
# 다양한 컬럼의 값들을 넣어서 확인할 수 있도록 발전시킬 수 있음
movie_name = input('영화 이름을 정확히 입력하세요 : ')
movie_find_count = int(input('비슷한 영화는 몇 개를 찾기를 원하세요(숫자를 입력하세요) : '))
find_sim_movie(data, sorted_genres_sim, movie_name, movie_find_count)

영화 이름을 정확히 입력하세요 : Sheena
비슷한 영화는 몇 개를 찾기를 원하세요(숫자를 입력하세요) : 10


,original_title,genres,popularity,runtime,vote_average,vote_count
859,Thunderbirds,Action Adventure Comedy Family Fantasy Science...,9.278750,95.0,4.2,91
2069,Teenage Mutant Ninja Turtles III,Action Adventure Comedy Family Fantasy Science...,10.207065,96.0,5.1,184
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
1936,Tank Girl,Action Comedy Fantasy Science Fiction,8.186936,98.0,5.5,138
342,Men in Black,Action Adventure Comedy Science Fiction,104.121555,98.0,6.9,4412
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800
